In [11]:
import numpy as np
import torch
import os
import sys

## 2 Deep Q-Learning

### 2.4 Basic Q-Learning

CartPole-v1

In [17]:
command = "python ../scripts/example_parse_tensorboard.py \
    -i ../../data/hw3_dqn_dqn_CartPole-v1_s64_l2_d0.99_07-10-2023_23-06-56/\
    -n 'dqn lunarlander' \
    -d eval_return \
"
os.system(command)

Traceback (most recent call last):
  File "/Users/admin/opt/anaconda3/envs/cs285/lib/python3.9/site-packages/tensorboard/backend/event_processing/directory_watcher.py", line 88, in Load
    for event in self._LoadInternal():
  File "/Users/admin/opt/anaconda3/envs/cs285/lib/python3.9/site-packages/tensorboard/backend/event_processing/directory_watcher.py", line 110, in _LoadInternal
    self._InitializeLoader()
  File "/Users/admin/opt/anaconda3/envs/cs285/lib/python3.9/site-packages/tensorboard/backend/event_processing/directory_watcher.py", line 173, in _InitializeLoader
    path = self._GetNextPath()
  File "/Users/admin/opt/anaconda3/envs/cs285/lib/python3.9/site-packages/tensorboard/backend/event_processing/directory_watcher.py", line 210, in _GetNextPath
    for path in io_wrapper.ListDirectoryAbsolute(self._directory)
  File "/Users/admin/opt/anaconda3/envs/cs285/lib/python3.9/site-packages/tensorboard/backend/event_processing/io_wrapper.py", line 78, in ListDirectoryAbsolute
  

256

LunarLander-v2

CartPole-v1, learning_rate = 0.05